In [7]:
import customtkinter # used for graphical representation
import time          # used to enhance the effect
import colorsys      # handeling colorformat 
#from colorutils import Color

customtkinter.set_appearance_mode("dark")

class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()
        #App Settings
        self.title("MMT Project")
        self.attributes('-fullscreen',True)
        self.open = True
        self.speed = 1
        self.running = False


        self.hsv= [13,62,45]
        self.color = "#733B2c"
        self.authors = "Henry Lorenz & Fabian Lampe"
        self.bgSat = 0
        self.bgColor = "#000000"
        self.config(bg = self.bgColor)

        self.uiColor = "#E86E4D"
        self.hoverColor = "#e24920"
        self.borderColor = "#372e29"
        #Elements

        # closeButton
        self.closeButton = customtkinter.CTkButton(
            master=self,
            text="X",
            width=20,
            height=20, 
            command=self.closeCallback,
            fg_color=self.uiColor,
            hover_color=self.hoverColor)
        # / closeButton

        # Orange Box
        self.orangeBox = customtkinter.CTkFrame(
            master=self,
            width=200,
            height=200,
            corner_radius=50,
            bg_color=self.bgColor,
            fg_color=self.color,
        )

        # Authorlabel
        self.authorsLabel = customtkinter.CTkLabel(
            master=self,
            text= " Authors: " + str(self.authors),
            height=20,
            text_color="gray",
            bg_color=self.bgColor)
        self.authorsLabel.cget("font").configure(size=12)

        # settingsFrame
        self.settingsFrame = customtkinter.CTkFrame(
            master=self,
            width=350,
            height=300,
            border_width=1,
            bg_color=self.bgColor,
            border_color=self.borderColor)

        self.settingsLabel = customtkinter.CTkLabel(
            master=self.settingsFrame,
            text="Settings",
            height=20,
            text_color="gray",
            bg_color="transparent")
        self.settingsLabel.cget("font").configure(size=16,weight="bold")

        # color Frame
        self.hsvFrame = customtkinter.CTkFrame(
            master=self.settingsFrame,
            width=335,
            height=150,
            fg_color="transparent",
            border_width=1,
            border_color=self.borderColor)

        self.hsvLabel = customtkinter.CTkLabel(
            master=self.hsvFrame,
            text="Color Settings",
            height=22,
            text_color="gray",
            bg_color="transparent")
        self.hsvLabel.cget("font").configure(size=14, slant="italic")

        self.hexLabel = customtkinter.CTkLabel(
            master=self.hsvFrame,
            text=str(self.rgb2hex(self.hsv2rgb(self.hsv))),
            height=22,
            text_color=self.color,
            bg_color="transparent")
        self.hexLabel.cget("font").configure(size=16, weight="bold")

        # speed Frame
        self.speedFrame = customtkinter.CTkFrame(
            master=self.settingsFrame,
            width=335,
            height=100,
            fg_color="transparent",
            border_width=1,
            border_color=self.borderColor)
        
        self.speedLabel = customtkinter.CTkLabel(
            master=self.speedFrame,
            text="Time Settings",
            height=22,
            text_color="gray",
            bg_color="transparent")
        self.speedLabel.cget("font").configure(size=14, slant="italic")


        # hsv Inputs
        self.h_slider = self.buildSlider(self.hsvFrame, "Hue", 0, 360, self.hsv[0],self.s_sliderCallback, enabled=False)
        self.s_slider = self.buildSlider(self.hsvFrame, "Saturation", 20, 100, self.hsv[1], self.s_sliderCallback)
        self.v_slider = self.buildSlider(self.hsvFrame, "Value", 10, 100, self.hsv[2], self.v_sliderCallback)
        self.bg_slider = self.buildSlider(self.hsvFrame, "BG Saturation", 0, 100, self.bgSat, self.bg_sliderCallback)

        self.hexLabel.place(relx=0.95,rely=0.05,anchor=customtkinter.NE)
        self.h_slider[0].place(relx=.25, rely=.35, anchor=customtkinter.SE)
        self.h_slider[1].place(relx=.34, rely=.35, anchor=customtkinter.SE)
        self.h_slider[2].place(relx=.38, rely=.325, anchor=customtkinter.SW)

        self.s_slider[0].place(relx=.25, rely=.575, anchor=customtkinter.SE)
        self.s_slider[1].place(relx=.34, rely=.575, anchor=customtkinter.SE)
        self.s_slider[2].place(relx=.38, rely=.55, anchor=customtkinter.SW)

        self.v_slider[0].place(relx=.25, rely=.7, anchor=customtkinter.SE)
        self.v_slider[1].place(relx=.34, rely=.7, anchor=customtkinter.SE)
        self.v_slider[2].place(relx=.38, rely=.775, anchor=customtkinter.SW)

        self.bg_slider[0].place(relx=.25, rely=.975, anchor=customtkinter.SE)
        self.bg_slider[1].place(relx=.34, rely=.975, anchor=customtkinter.SE)
        self.bg_slider[2].place(relx=.38, rely=.95, anchor=customtkinter.SW)

        # time Inputs
        self.time_slider = self.buildSlider(self.speedFrame, "Updaterate", 0.5, 2, self.speed, self.time_sliderCallback, "float")
        self.time_button =customtkinter.CTkButton(
            master= self.speedFrame,
            text="▶",
            width=45,
            height=45, 
            command=self.pauseCallback,
            fg_color="#E86E4D",
            hover_color="#e24920"
        )
        self.time_button.cget("font").configure(size=20, weight="bold")

        self.time_slider[0].place(relx=.25, rely=.45, anchor=customtkinter.SE)
        self.time_slider[1].place(relx=.34, rely=.45, anchor=customtkinter.SE)
        self.time_slider[2].place(relx=.38, rely=.425, anchor=customtkinter.SW)

        self.time_button.place(relx=.24,rely=0.93,anchor=customtkinter.SE)

        # /settingsFrame

        #Place Elements
        #general Elements
        self.closeButton.place(relx = 1, rely = 0,anchor=customtkinter.NE)
        self.authorsLabel.place(relx = 0, rely = 1, anchor=customtkinter.SW)
        self.orangeBox.place(relx=0.5, rely=0.5, anchor=customtkinter.CENTER)

        #hsvFrame Elements
        self.hsvFrame.place(relx=.02,rely=.1,anchor=customtkinter.NW)
        self.hsvLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)

        self.speedFrame.place(relx=.02,rely=.99,anchor=customtkinter.SW)
        self.speedLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)
        

        #Setting Elements
        self.settingsFrame.place(relx=1, rely=1, anchor=customtkinter.SE)
        self.settingsLabel.place(relx=.01, rely=.01, anchor=customtkinter.NW)

# Callback Methodes
    def closeCallback(self):
        self.open = False
        self.destroy()

    def s_sliderCallback(self,value):
        self.hsv[1] = value
        self.s_slider[1].configure(text="{0:>2}".format(str(round(self.hsv[1]))))
        self.color=str(self.rgb2hex(self.hsv2rgb(self.hsv)))
        self.hexLabel.configure(text="{}".format(self.color), text_color=self.color)
        self.orangeBox.configure(fg_color=self.color)
    
    def v_sliderCallback(self,value):
        self.hsv[2] = value
        self.v_slider[1].configure(text="{0:>2}".format(str(round(self.hsv[2]))))
        self.color=str(self.rgb2hex(self.hsv2rgb(self.hsv)))
        self.hexLabel.configure(text="{}".format(self.color), text_color=self.color)
        self.orangeBox.configure(fg_color=self.color)

    def time_sliderCallback(self,value):
        self.speed = value
        self.time_slider[1].configure(text="{0:>2}".format(str(round(self.speed,1))))

    def bg_sliderCallback(self,value):
        self.bgSat = value
        self.bg_slider[1].configure(text="{0:>2}".format(str(round(self.bgSat,1))))
        self.bgColor = self.rgb2hex(self.hsv2rgb([0,0,self.bgSat]))
        self.config(bg=self.bgColor)
    
    def pauseCallback(self):
        if(self.running):
            self.running = False
            self.time_button.configure(text="▶")
        else:
            self.running = True
            self.time_button.configure(text="⏸")

# Layout Methodes
    def buildSlider(self, origin, name, start, end, value, callback, enabled=True):

        if(enabled):
            Slider = customtkinter.CTkSlider(
                master = origin,
                from_=start, 
                to=end,
                button_color="#E86E4D",
                button_hover_color="#e24920",
                progress_color="#f8d3c9",
                hover=True,
                command=callback)
            
        else:
            Slider = customtkinter.CTkSlider(
                master = origin,
                from_=start, 
                to=end,
                button_color="#555555",
                progress_color="#999999",
                state="disabled",
                command=callback)
        
        Slider.set(value)

        Label = customtkinter.CTkLabel(
            master=origin,
            text=name,
            height=20,
            text_color="gray",
            bg_color="transparent")
        Label.cget("font").configure(size=12)
        
        Value = customtkinter.CTkLabel(
            master=origin,
            text= str(value),
            height=20,
            text_color="gray",
            bg_color="transparent")
        Value.cget("font").configure(size=12)

        return [Label, Value, Slider]

# Utility Methods
    def rgb2hex(self,rgb):
            r,g,b = round(rgb[0]),round(rgb[1]),round(rgb[2])
            val = "#{:02X}{:02X}{:02X}".format(r,g,b)
            return val

    def hex2rgb(self,hex):
            h = hex.strip("#") 
            rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
            return rgb
        
    def rgb2hsv(self,rgb):
            r,g,b = (rgb[0]/255),(rgb[1]/255),(rgb[2]/255)
            # Constraining the values to the range 0 to 1
            # defining the following terms for convenience
            Cmax = max(r, g, b)
            Cmin = min(r, g, b)
            delta = Cmax - Cmin
            # hue calculation
            if (delta == 0):  H = 0
            elif (Cmax == r): H = (60 * (((g  - b) / delta) % 6))
            elif (Cmax == g): H = (60 * (((b  - r) / delta) + 2))
            elif (Cmax == b): H = (60 * (((r  - g) / delta) + 4))
            
            # saturation calculation
            if (Cmax == 0): S = 0 
            else: S = delta / Cmax
            # value calculation
            V = Cmax 
            return(round(H),round(S*100),round(V*100))

    def hsv2rgb(self,hsv):
            print(hsv)
            h,s,v = (hsv[0]/360),(hsv[1]/100),(hsv[2]/100)
            print(h,s,v)
            rgb= colorsys.hsv_to_rgb(h,s,v)
            return (round(rgb[0]*255),round(rgb[1]*255),round(rgb[2]*255))


In [8]:
if __name__ == "__main__":
    app = App()
    #app.mainloop()

    time_1 = 0

    while 1:
        time_2 = time.process_time()
        app.update_idletasks()
        app.update()       
        if app.running and (time_2 - time_1 > app.speed):
            if app.cget("background") == "black":
                app.config(bg = "white")
                app.authorsLabel.configure(bg_color="white")
                app.orangeBox.configure(bg_color="white")
            else:
                app.config(bg = "black")
                app.authorsLabel.configure(bg_color="black")
                app.orangeBox.configure(bg_color="black")
            time_1 = time_2
        if(app.open == False):
            break

[13, 62, 45]
0.03611111111111111 0.62 0.45


invalid command name "139629966015104_click_animation"
    while executing
"139629966015104_click_animation"
    ("after" script)


[0, 3.0, 0]
0.0 0.03 0.0
[0, 3.5000000000000004, 0]
0.0 0.035 0.0
[0, 3.5000000000000004, 0]
0.0 0.035 0.0
[0, 5.0, 0]
0.0 0.05 0.0
[0, 6.5, 0]
0.0 0.065 0.0
[0, 8.0, 0]
0.0 0.08 0.0
[0, 8.5, 0]
0.0 0.085 0.0
[0, 9.5, 0]
0.0 0.095 0.0
[0, 11.5, 0]
0.0 0.115 0.0
[0, 12.5, 0]
0.0 0.125 0.0
[0, 14.000000000000002, 0]
0.0 0.14 0.0
[0, 15.0, 0]
0.0 0.15 0.0
[0, 16.0, 0]
0.0 0.16 0.0
[0, 16.5, 0]
0.0 0.165 0.0
[0, 17.0, 0]
0.0 0.17 0.0
[0, 17.5, 0]
0.0 0.175 0.0
[0, 18.0, 0]
0.0 0.18 0.0
[0, 18.5, 0]
0.0 0.185 0.0
[0, 19.0, 0]
0.0 0.19 0.0
[0, 19.5, 0]
0.0 0.195 0.0
[0, 20.0, 0]
0.0 0.2 0.0
[0, 21.0, 0]
0.0 0.21 0.0
[0, 21.5, 0]
0.0 0.215 0.0
[0, 22.0, 0]
0.0 0.22 0.0
[0, 23.0, 0]
0.0 0.23 0.0
[0, 23.5, 0]
0.0 0.235 0.0
[0, 24.0, 0]
0.0 0.24 0.0
[0, 24.5, 0]
0.0 0.245 0.0
[0, 25.5, 0]
0.0 0.255 0.0
[0, 26.5, 0]
0.0 0.265 0.0
[0, 27.0, 0]
0.0 0.27 0.0
[0, 27.500000000000004, 0]
0.0 0.275 0.0
[0, 28.499999999999996, 0]
0.0 0.285 0.0
[0, 28.999999999999996, 0]
0.0 0.29 0.0
[0, 30.0, 0]
0.0 0.3 

In [9]:
import colorsys 
def rgb2Hex(rgb):
        r,g,b = round(rgb[0]),round(rgb[1]),round(rgb[2])
        val = "#{:02X}{:02X}{:02X}".format(r,g,b)
        return val

def hex2rgb(hex):
        h = hex.strip("#") 
        rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
        return rgb
    
def rgb2hsv(rgb):
        r,g,b = (rgb[0]/255),(rgb[1]/255),(rgb[2]/255)
   # Constraining the values to the range 0 to 1
# defining the following terms for convenience
        Cmax = max(r, g, b)
        Cmin = min(r, g, b)
        delta = Cmax - Cmin
        # hue calculation
        if (delta == 0):
                H = 0
        elif (Cmax == r):
                H = (60 * (((g  - b) / delta) % 6))
        elif (Cmax == g):
                H = (60 * (((b  - r) / delta) + 2))
        elif (Cmax == b):
                H = (60 * (((r  - g) / delta) + 4))
        
        # saturation calculation
        if (Cmax == 0):
                S = 0 
        else:
                S = delta / Cmax
        # value calculation
        V = Cmax 
        return(round(H),round(S*100),round(V*100))

def hsv2rgb(hsv):
        print(hsv)
        h,s,v = (hsv[0]/360),(hsv[1]/100),(hsv[2]/100)
        print(h,s,v)
        rgb= colorsys.hsv_to_rgb(h,s,v)
        return (round(rgb[0]*255),round(rgb[1]*255),round(rgb[2]*255))


hex2rgb("#E86E4D")
rgb2hsv((232, 110, 77))
hsv2rgb((13,67,91))
rgb2Hex((232, 110, 77))


(13, 67, 91)
0.03611111111111111 0.67 0.91


'#E86E4D'